### Step 1: Loading and Reading the Data

In [245]:
# Dependencies and setup
import pandas as pd
import re
from datetime import datetime

# Files to Load
persons_data_to_load = "persons_data.csv"
inventory_data_to_load = "inventory_data.csv"
majors_data_to_load = "majors_data.csv"
occupancy_data_to_load = "occupancy_data.csv"

# Reading persons, inventory, majors, and occupancy data files and storing them into Pandas DataFrames
persons_data = pd.read_csv(persons_data_to_load)
majors_data = pd.read_csv(majors_data_to_load)
inventory_data = pd.read_csv(inventory_data_to_load)
occupancy_data = pd.read_csv(occupancy_data_to_load)

# Viewing the first five rows of each DataFrame
persons_data.head()
# majors_data.head()
# inventory_data.head()
# occupancy_data.head()

,personId,firstName,lastName,email,dob,address,majors
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David,Taylor,david.taylor@example.com,1992-08-15,"767 Walnut St, Georgetown, OH",Communications
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane,Wilson,jane.wilson@example.com,1969-08-20,"168 Birch St, Riverside, FL","Music, Finance"
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John,Brown,john.brown@example.com,1967-12-14,"726 Maple St, Fairview, PA",NaN
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda,Taylor,linda.taylor@example.com,1983-12-04,"171 Spruce St, Riverside, TX","Social Work, Anthropology"
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah,Doe,sarah.doe@example.com,1986-08-31,"987 Walnut St, Georgetown, OH","Psychology, Criminal Justice, History, Foreign..."


### Step 2: Cleaning and Validating the `persons_data` DataFrame

In [246]:
# Checking for duplicate entries in persons_data based on the 'personId' field
duplicate_persons = persons_data[persons_data.duplicated(subset='personId', keep=False)]

# Checking if the duplicate_persons DataFrame is empty
if duplicate_persons.empty:
    print("There are no duplicate personId entries in persons_data.")
else:
    print("There are duplicate personId entries in persons_data:")
    print(duplicate_persons)

There are no duplicate personId entries in persons_data.


In [247]:
# Checking for unique email addresses
duplicate_emails = persons_data[persons_data.duplicated(subset='email', keep=False)]

# Dropping the rows with non-unique repeated email addresses
persons_data_unique_emails = persons_data.drop_duplicates(subset='email', keep=False)

# Displaying the results
print("Rows with duplicate email addresses:")
print(duplicate_emails)
print("\nDataFrame with unique email addresses:")
print(persons_data_unique_emails)

Rows with duplicate email addresses:
                                  personId firstName  lastName  \
0     7b1fb398-72d7-421e-808c-631a427bfbc4     David    Taylor   
1     99a4b233-2804-420b-b757-039f0f3cceb7      Jane    Wilson   
2     cc3b2df2-a593-493e-af82-1f0d11c7439b      John     Brown   
3     cb669177-4020-4284-a3dc-9a0c312411d4     Linda    Taylor   
4     ba1443a2-dfeb-441b-b41d-078b36416704     Sarah       Doe   
...                                    ...       ...       ...   
4995  bc90a95b-9d36-4905-aa0f-636f5c1e4216      Jane    Miller   
4996  499eca03-0f5c-4026-859e-cd007c591ae9       Bob       Doe   
4997  b486891f-da16-452a-a27b-209a071c2f10     Sarah  Williams   
4998  f7672679-6fbf-43ae-8593-cb267ab35a56      John    Wilson   
4999  bcd7282a-5497-43bc-88fd-5b5ec3b0e5ce     Emily     Smith   

                           email         dob                        address  \
0       david.taylor@example.com  1992-08-15  767 Walnut St, Georgetown, OH   
1        jan

**Remark:** The above poses a problem as we have the same email addresses being mapped to different people with unique IDs. There are a few options as to how we can handle this: (1) Generate a unique email for each person by appending a number at the end before '@' (this could be problematic though since we do not know if these emails have already been assigned to such users); (2) Completely drop the email column from our final DataFrame; or (3) Keep as is and assume "example" is a placeholder. I will provide the code for all three options below, but am choosing option (3) for compiling the final DataFrame.  

In [156]:
# (1) Function to generate unique email addresses
#def generate_unique_emails(df):
    email_counter = {}
    
    for idx, row in df.iterrows():
        email = row['email']
        base_email = email.split('@')[0]
        domain = email.split('@')[1]
        
        # If the email is already seen, append a number to make it unique
       if email in email_counter:
            #email_counter[email] += 1
            #new_email = f"{base_email}{email_counter[email]}@{domain}"
        else:
           # email_counter[email] = 0
           # new_email = email
        
        # Update the email in the DataFrame
        df.at[idx, 'email'] = new_email
    
    return df

# Generating the unique email addresses by applying defined function above
persons_data_unique_emails = generate_unique_emails(persons_data)

print("DataFrame with unique email addresses:")
print(persons_data_unique_emails)


# (2) Dropping the email column from the DataFrame entirely
if 'email' in persons_data.columns:
    persons_data = persons_data.drop(columns=['email'])


In [248]:
# Checking for null entries in the personId column
null_personIds = persons_data[persons_data['personId'].isnull()]

if null_personIds.empty:
    print("There are no null entries for personId in persons_data.")
else:
    print("Null entries found for personId:")
    print(null_personIds)

There are no null entries for personId in persons_data.


In [249]:
# Checking for repeated addresses
repeated_addresses = persons_data[persons_data.duplicated(subset='address', keep=False)]

print("Rows with repeated addresses:")
print(repeated_addresses)

Rows with repeated addresses:
                                  personId firstName  lastName  \
62    4356331d-4657-4a7b-bbad-d88800bceffc     Sarah     Smith   
66    0d92bcb1-f7b2-4c94-875c-1c0c250f7aec     David   Johnson   
84    98c695fe-e802-4df1-9739-285c0e475e5a     David       Doe   
187   88e05673-ef48-442e-a342-78d9db536658     James   Johnson   
220   cac26b36-cc2d-4032-8258-784a90771b3e      John  Williams   
...                                    ...       ...       ...   
4768  b35d022b-2ea0-40c0-9530-8de067e26d60       Bob       Doe   
4803  e89e909d-dace-4165-a48f-dd3c0f5f1c5b     Alice    Taylor   
4814  8ed4d40c-ff5d-4d66-958f-66b87be9b4a3     Alice     Moore   
4827  0e26118f-6c3d-4f63-b495-af9a0270f492       Bob     Moore   
4981  6e315a8d-6c35-4d0b-91c4-8bbc1f63f5ad     Alice     Davis   

                          email         dob                        address  \
62      sarah.smith@example.com  1978-12-17   477 Cedar St, Georgetown, TX   
66    david.johnson@e

**Remark:** Addresses need not be unique, however, would be more descriptive if they contained the zip code and/or a second address. This is something I would address (pun intended) with the individual responsible for compiling the data source.

In [250]:
# Ensuring the uniqueness of the email and personID fields
persons_data = persons_data.drop_duplicates(subset=['personId', 'email'])

# Replace NaNs with empty strings for optional fields
persons_data.fillna('', inplace=True)

# Validating email format
email_format = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'  # Regular expression pattern used to validate email addresses
persons_data = persons_data[persons_data['email'].str.match(email_format)]  # Filter to keep only the rows where the 'email' column matches the regular expression above

# Validating dob format
persons_data['dob'] = pd.to_datetime(persons_data['dob'], errors='coerce')  # Convert the 'dob' column to datetime format
persons_data = persons_data[persons_data['dob'].notna()]  # Keep only the rows where the 'dob' column is not Not a Time


# Defining a function to split the address string into the desired components (i.e., address1, address2, city, state, and zip)
# From a quick look at the persons_data CSV file, the addresses seem to be missing zip codes and a second address component. Thus, these addresses do not form full address strings and we need to go back to the source to obtain these missing pieces of information.
# However, we can still implement a function that can split the address string with or without these two components, which easily allows us to update the final DataFrame once the complete data is obtained.
def split_address(address):
    if pd.isna(address):
        return pd.Series([None, None, None, None, None]) # Checking if the address is a missing value. If it is, a series with five 'None' values will be returned.
    

    # Splitting the address string into parts using a comma as the delimiter
    # This check ensures that the address has at least three components: main address, city, and state/ZIP. If not, a series with five 'None' values will be returned.
    parts = address.split(',')
    if len(parts) < 3:
        return pd.Series([None, None, None, None, None]) 
    
    # Extracting the address components
    address_part = parts[0].strip()
    
    # Checking if there is a second address part (e.g., apartment or unit number)
    if 'apt' in address_part.lower() or 'unit' in address_part.lower() or '#' in address_part:
        address_parts = address_part.split()
        second_address_part = address_parts[-1].strip()
        address_part = ' '.join(address_parts[:-1]).strip()
    else:
        second_address_part = None
    
    city_part = parts[1].strip()
    state_zip_part = parts[2].strip().split()
    
    if len(state_zip_part) == 1:
        state_part = state_zip_part[0].strip()
        zip_part = None
    elif len(state_zip_part) == 2: 
        state_part = state_zip_part[0].strip()
        zip_part = state_zip_part[1].strip()
    else:
        return pd.Series([None, None, None, None, None])
    
    return pd.Series([address_part, second_address_part, city_part, state_part, zip_part])


# Applying the function to the address column of the DataFrame
persons_data[['address1', 'address2', 'city', 'state', 'zip']] = persons_data['address'].apply(split_address)

# Dropping the original address column of the DataFrame
persons_data.drop(columns=['address'], inplace=True)

# Concatenating the first and last names to make-up Full Name
persons_data['name'] = persons_data['firstName'] + ' ' + persons_data['lastName']

# Selecting the desired columns from the cleaned DataFrame
persons_data2 = persons_data[['personId', 'name', 'email', 'dob', 'address1', 'address2', 'city', 'state', 'zip', 'majors']]
persons_data2.head()

,personId,name,email,dob,address1,address2,city,state,zip,majors
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David Taylor,david.taylor@example.com,1992-08-15,767 Walnut St,None,Georgetown,OH,None,Communications
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane Wilson,jane.wilson@example.com,1969-08-20,168 Birch St,None,Riverside,FL,None,"Music, Finance"
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John Brown,john.brown@example.com,1967-12-14,726 Maple St,None,Fairview,PA,None,
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda Taylor,linda.taylor@example.com,1983-12-04,171 Spruce St,None,Riverside,TX,None,"Social Work, Anthropology"
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah Doe,sarah.doe@example.com,1986-08-31,987 Walnut St,None,Georgetown,OH,None,"Psychology, Criminal Justice, History, Foreign..."


### Step 3: Merging `occupancy_data` and `inventory_data` DataFrames

In [251]:
# Merge occupancy_data with inventory_data to get bedId
occupancy_with_bedId = pd.merge(occupancy_data, inventory_data, on=['buildingName', 'roomName', 'bedName'], how='left')

# Now merge with persons_data using the updated occupancy_with_bedId
merged_data = pd.merge(persons_data2, occupancy_with_bedId, on='personId', how='left')

# Drop unnecessary columns and keep the required ones
merged_data = merged_data[['personId', 'name', 'email', 'dob', 'address1', 'address2', 'city', 'state', 'zip', 'majors', 'bedId']]

merged_data.head()

,personId,name,email,dob,address1,address2,city,state,zip,majors,bedId
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David Taylor,david.taylor@example.com,1992-08-15,767 Walnut St,None,Georgetown,OH,None,Communications,NaN
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane Wilson,jane.wilson@example.com,1969-08-20,168 Birch St,None,Riverside,FL,None,"Music, Finance",NaN
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John Brown,john.brown@example.com,1967-12-14,726 Maple St,None,Fairview,PA,None,,a32b34cf-0356-4739-ab43-4a9ed70818f8
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda Taylor,linda.taylor@example.com,1983-12-04,171 Spruce St,None,Riverside,TX,None,"Social Work, Anthropology",01f250f6-a3a3-492b-ba52-3f53f4483316
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah Doe,sarah.doe@example.com,1986-08-31,987 Walnut St,None,Georgetown,OH,None,"Psychology, Criminal Justice, History, Foreign...",f8ef5ea9-e2bf-4eda-9d7a-c7cdf30dc649


### Step 4: Cleaning and Validating `majors_data` DataFrame

In [252]:
# Checking for duplicate major entries by name
duplicate_majors = majors_data[majors_data.duplicated(subset='name', keep=False)]

# Displaying the duplicate entries
print("Duplicate entries by name in majors_data:")
print(duplicate_majors)

# Checking for duplicate major entries by id
duplicate_majors = majors_data[majors_data.duplicated(subset='id', keep=False)]

# Displaying the duplicate entries
print("Duplicate entries by id in majors_data:")
print(duplicate_majors)

# Checking for duplicate major entries by displayId
duplicate_majors = majors_data[majors_data.duplicated(subset='displayId', keep=False)]

# Displaying the duplicate entries
print("Duplicate entries by displayID in majors_data:")
print(duplicate_majors)

Duplicate entries by name in majors_data:
                                      id  \
0   12c5d420-9bbc-455d-9794-7780bc5fea5b   
1   d3d06262-f539-4a05-9e71-6e6113e9529b   
2   19a8bee2-7566-411a-9919-61d44ccc81ef   
3   b33ae65d-523e-48c6-b10e-59346e1c6523   
4   3c0ebd74-4105-4532-a321-39bc3d6ab3d8   
..                                   ...   
95  1e0b04ba-18d5-4ab9-a1e5-23449bbc4a0b   
96  d25284f4-c84e-48db-b41e-dc42085a55f9   
97  1f2a78f5-deae-438d-be76-6d733a7dac87   
98  c6f0e3ea-fc4a-4768-a155-f0bf0b943717   
99  62f9fcfb-92d6-4bc8-96ff-e8847c7b27ae   

                               displayId                     name  \
0   d24fcc15-c1ac-46b1-aca8-dc998cf45e78         Computer Science   
1   1e4c7f9f-40f5-4032-bd44-7036952e3105   Mechanical Engineering   
2   c84821a7-9878-46e4-a666-34a538c10aa2   Electrical Engineering   
3   d92604c6-1626-4794-9cce-cf0a7e136c19                  Biology   
4   7468a134-27ba-4c1e-8b24-8ff51cecfb5c                Chemistry   
..             

**Remark:** As shown in the output of the above block of code, we have duplicate entries of major names, with each having a different id and displayID associated with it (e.g., Mathematics appears several times in the DataFrame, each time having it associated with a different id and displayID). Since we have multiple ids mapping to the same major name, we will deal with this issue by choosing the first of the ids for each of them.

In [253]:
# Dropping duplicate entries in majors_data DataFrame by name, keeping only the first occurrence
unique_majors_data = majors_data.drop_duplicates(subset='name', keep='first')

# Displaying the first few rows of the unique majors data
print("Unique majors data:")
unique_majors_data.head()

Unique majors data:


,id,displayId,name,description
0,12c5d420-9bbc-455d-9794-7780bc5fea5b,d24fcc15-c1ac-46b1-aca8-dc998cf45e78,Computer Science,The study of computers and computational systems.
1,d3d06262-f539-4a05-9e71-6e6113e9529b,1e4c7f9f-40f5-4032-bd44-7036952e3105,Mechanical Engineering,The design and manufacture of mechanical systems.
2,19a8bee2-7566-411a-9919-61d44ccc81ef,c84821a7-9878-46e4-a666-34a538c10aa2,Electrical Engineering,The study of electricity and its applications.
3,b33ae65d-523e-48c6-b10e-59346e1c6523,d92604c6-1626-4794-9cce-cf0a7e136c19,Biology,The study of living organisms.
4,3c0ebd74-4105-4532-a321-39bc3d6ab3d8,7468a134-27ba-4c1e-8b24-8ff51cecfb5c,Chemistry,The study of substances and their properties.


In [254]:
# Addding majorIds by merging with majors_data
major_dict = pd.Series(unique_majors_data.name.values, index=unique_majors_data.id).to_dict()

# Function to map the major names to their corresponding ids
def map_majors_to_ids(majors):
    if not majors:
        return ''
    major_list = majors.split(', ')
    major_ids = [k for k, v in major_dict.items() if v in major_list]
    return ', '.join(major_ids)

merged_data['majorIds'] = merged_data['majors'].apply(map_majors_to_ids)

# Dropping the 'majors' column 
final_data = merged_data.drop(columns=['majors'])

# Displaying the first five rows of the updated DataFrame for verification purposes
final_data.head()

,personId,name,email,dob,address1,address2,city,state,zip,bedId,majorIds
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David Taylor,david.taylor@example.com,1992-08-15,767 Walnut St,None,Georgetown,OH,None,NaN,2c6e4248-da4d-4cb3-ac58-3cd04aa203fa
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane Wilson,jane.wilson@example.com,1969-08-20,168 Birch St,None,Riverside,FL,None,NaN,"49f57722-52fb-47b6-8e1e-54de177e0d19, e00c66ff..."
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John Brown,john.brown@example.com,1967-12-14,726 Maple St,None,Fairview,PA,None,a32b34cf-0356-4739-ab43-4a9ed70818f8,
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda Taylor,linda.taylor@example.com,1983-12-04,171 Spruce St,None,Riverside,TX,None,01f250f6-a3a3-492b-ba52-3f53f4483316,"37b765f0-fc71-4c9c-838b-f3a5ce0706ff, 8402b4cd..."
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah Doe,sarah.doe@example.com,1986-08-31,987 Walnut St,None,Georgetown,OH,None,f8ef5ea9-e2bf-4eda-9d7a-c7cdf30dc649,"4d0b9e50-43d1-4da0-af2b-97d510bab9e2, f6cf2da9..."


### Step 5: Final Cleaning and Saving of Merged DataFrame

In [266]:
# Selecting and reordering desired columns to match that in the question
final_data1 = final_data[['personId', 'name', 'email', 'dob', 'address1', 'address2', 'city', 'state', 'zip', 'majorIds', 'bedId']]

# Converting the 'None' string to NaN
final_data1.replace('None', pd.NA, inplace=True)

# Identifing the rows with missing required data (i.e., missing majorIds and/or bedIDs since all rows are missing address2 and zip codes)
missing_data_rows = final_data1[(final_data1['majorIds'] == '') | (final_data1['bedId'] == '')]

# Saving these rows to a CSV file for further inspection
missing_data_file = 'missing_data_rows.csv'
missing_data_rows.to_csv(missing_data_file, index=False)

# Saving final_data1 DataFrame into a CSV file
final_data_file = 'final_data1.csv'
final_data1.to_csv(final_data_file, index=False)

# Displaying the first five rows of the missing_data_file DataFrame for verification purposes
#missing_data_file.head()

# Displaying the first five rows of the final_data1 DataFrame for verification purposes
final_data1.head()

,personId,name,email,dob,address1,address2,city,state,zip,majorIds,bedId
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David Taylor,david.taylor@example.com,1992-08-15,767 Walnut St,None,Georgetown,OH,None,2c6e4248-da4d-4cb3-ac58-3cd04aa203fa,NaN
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane Wilson,jane.wilson@example.com,1969-08-20,168 Birch St,None,Riverside,FL,None,"49f57722-52fb-47b6-8e1e-54de177e0d19, e00c66ff...",NaN
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John Brown,john.brown@example.com,1967-12-14,726 Maple St,None,Fairview,PA,None,,a32b34cf-0356-4739-ab43-4a9ed70818f8
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda Taylor,linda.taylor@example.com,1983-12-04,171 Spruce St,None,Riverside,TX,None,"37b765f0-fc71-4c9c-838b-f3a5ce0706ff, 8402b4cd...",01f250f6-a3a3-492b-ba52-3f53f4483316
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah Doe,sarah.doe@example.com,1986-08-31,987 Walnut St,None,Georgetown,OH,None,"4d0b9e50-43d1-4da0-af2b-97d510bab9e2, f6cf2da9...",f8ef5ea9-e2bf-4eda-9d7a-c7cdf30dc649
